# Training a Machine Learning Model in Spark

## Connect to Spark

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# SparkSession
URL_SPARK = "spark://spark:7077"

spark = (
    SparkSession.builder
    .appName("spark-ml")
    .master(URL_SPARK)
    .getOrCreate()
)

## Load Data

In [4]:
df_avocado = spark.read.csv("/data/avocado.csv", header=True, inferSchema=True)
# cache data
df_avocado.cache()
df_avocado.show(4)

+---+-------------------+------------+------------+-------+---------+-----+----------+----------+----------+-----------+------------+----+------+
|_c0|               Date|AveragePrice|Total Volume|   4046|     4225| 4770|Total Bags|Small Bags|Large Bags|XLarge Bags|        type|year|region|
+---+-------------------+------------+------------+-------+---------+-----+----------+----------+----------+-----------+------------+----+------+
|  0|2015-12-27 00:00:00|        1.33|    64236.62|1036.74| 54454.85|48.16|   8696.87|   8603.62|     93.25|        0.0|conventional|2015|Albany|
|  1|2015-12-20 00:00:00|        1.35|    54876.98| 674.28| 44638.81|58.33|   9505.56|   9408.07|     97.49|        0.0|conventional|2015|Albany|
|  2|2015-12-13 00:00:00|        0.93|   118220.22|  794.7|109149.67|130.5|   8145.35|   8042.21|    103.14|        0.0|conventional|2015|Albany|
|  3|2015-12-06 00:00:00|        1.08|    78992.15| 1132.0| 71976.41|72.58|   5811.16|    5677.4|    133.76|        0.0|conv

## Preprocess data

In [5]:
# Import machine learning libraries
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Machine learning pipeline
from pyspark.ml import Pipeline

# Import SQL transformer
from pyspark.ml.feature import SQLTransformer

In [6]:
df_avocado_train, df_avocado_test = df_avocado.randomSplit([0.75, 0.25], seed=214)

**Selecting columns with SQLTransformer**

This is a quite powerful transformer, which allows you to select and transform columns using SQL syntax.

In [13]:
COLUMNS = ['AveragePrice', '4225', '4770', 'Small Bags', 'Large Bags', 'XLarge Bags', 'type']
COLUMNS = [f"`{col}`" for col in COLUMNS]

sqlTransformer = SQLTransformer(
    statement=f"""
    
    SELECT
    {', '.join(COLUMNS)}
    ,YEAR(__THIS__.Date) AS year
    ,MONTH(__THIS__.Date) AS month

    FROM __THIS__

    """
)

# Visualize the data
sqlTransformer.transform(df_avocado_train).show(4)

+------------+---------+--------+----------+----------+-----------+------------+----+-----+
|AveragePrice|     4225|    4770|Small Bags|Large Bags|XLarge Bags|        type|year|month|
+------------+---------+--------+----------+----------+-----------+------------+----+-----+
|        0.49|286858.37|11642.46|  70749.02|  30142.78|        0.0|conventional|2015|   12|
|        0.71|141599.36|15486.97| 123158.22|  33065.33|    11190.0|conventional|2015|   12|
|         0.8|276556.76|84912.97| 136560.04|   12277.7|   15657.53|conventional|2015|   12|
|         0.8|455203.42|86202.11| 722787.61|  74359.03|   11407.39|conventional|2015|   12|
+------------+---------+--------+----------+----------+-----------+------------+----+-----+
only showing top 4 rows

